# c) Ngonye Falls Flow Analysis

Load the synthetic historic daily flow series for Ngonye and produce various summary statistics for later presentation.

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| ngonye_synthetic.csv  | Notebook: b_synthetic_flow_ngonye |Synthetic daily flow series for Ngonye  Falls 1924/25 - 2016/17  |
| selected_years.csv | Mott MacDonald - Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D | List of representative selected years |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
| ngonye_flow_daily.csv          | Daily flow data  |
| ngonye_flow_monthly.csv        | Flow summaries by month  |
| ngonye_flow_yearly.csv         | Flow summaries by year  |
| ngonye_flow_calmonthly.csv     | Flow summaries by calendar month |
| ngonye_flow_selected_years.csv | Flow summaries for selected representative years  |



## Parameters

In [2]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [3]:
import numpy as np
import pandas as pd

## Load the Daily Data

In [4]:
daily = pd.read_csv(output_data + "ngonye_synthetic.csv")
daily.tail(4)

,Date,LaggedDate,VicFalls,Conversion,Flow,Exceedance
33964,2017-09-27,2017-10-08,204.0,0.995184,203.017496,0.950
33965,2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950
33966,2017-09-29,2017-10-10,204.0,0.995184,203.017496,0.950
33967,2017-09-30,2017-10-11,201.0,0.986217,198.229576,0.958


Index by date and add some other columns for later use. 

Add a column for *WaterYear* which starts on 1st October and runs to 31st September the following year.

In [5]:
daily['Date']=pd.to_datetime(daily['Date'],format="%Y-%m-%d")#"%d/%m/%Y")
daily=daily.set_index(pd.DatetimeIndex(daily['Date']))


In [6]:
daily['Year']=daily.index.year
daily['Month']=daily.index.month
daily['Day']=daily.index.day
daily['MonthId']=daily['Year']+daily['Month']/100
daily['WaterYear']=daily.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
daily['WaterMonth']=daily.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)
daily['WaterDay']=daily.apply(lambda x: (x['Date']-pd.Timestamp(x['WaterYear'], 10, 1)).days+1,axis=1)
daily['WaterWeek']=np.floor((daily['WaterDay']-1)/7)+1
daily['Volume']=daily['Flow']*60*60*24/(1000*1000*1000)
daily=daily.astype({'WaterWeek': 'int32'})
daily=daily.drop('Date',axis=1)
daily.head(8)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,WaterWeek,Volume
Date,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124,0.999,1924,10,1,1924.1,1924,1,1,1,0.009602
1924-10-02,1924-10-13,100.0,1.111331,111.133124,0.999,1924,10,2,1924.1,1924,1,2,1,0.009602
1924-10-03,1924-10-14,100.0,1.111331,111.133124,0.999,1924,10,3,1924.1,1924,1,3,1,0.009602
1924-10-04,1924-10-15,100.0,1.111331,111.133124,0.999,1924,10,4,1924.1,1924,1,4,1,0.009602
1924-10-05,1924-10-16,100.0,1.111331,111.133124,0.999,1924,10,5,1924.1,1924,1,5,1,0.009602
1924-10-06,1924-10-17,100.0,1.111331,111.133124,0.999,1924,10,6,1924.1,1924,1,6,1,0.009602
1924-10-07,1924-10-18,100.0,1.111331,111.133124,0.999,1924,10,7,1924.1,1924,1,7,1,0.009602
1924-10-08,1924-10-19,100.0,1.111331,111.133124,0.999,1924,10,8,1924.1,1924,1,8,2,0.009602


## Setup the Monthly Data

Load the monthly data.

In [7]:

monthly=daily.groupby(['MonthId','Year','Month']).size().to_frame(name="Days").reset_index(['Month','Year'])
monthly

,Year,Month,Days
MonthId,,,
1924.10,1924,10,31
1924.11,1924,11,30
1924.12,1924,12,31
1925.01,1925,1,31
1925.02,1925,2,28
...,...,...,...
2017.05,2017,5,31
2017.06,2017,6,30
2017.07,2017,7,31


Set the index and add additional columns for later use.

In [8]:

monthly['Day']=1
monthly['DateStart']=pd.to_datetime(monthly[['Year','Month','Day']])
monthly=monthly.drop('Day',1)


monthly['WaterYear']=monthly.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
monthly['WaterMonth']=monthly.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)


monthly

,Year,Month,Days,DateStart,WaterYear,WaterMonth
MonthId,,,,,,
1924.10,1924,10,31,1924-10-01,1924,1
1924.11,1924,11,30,1924-11-01,1924,2
1924.12,1924,12,31,1924-12-01,1924,3
1925.01,1925,1,31,1925-01-01,1924,4
1925.02,1925,2,28,1925-02-01,1924,5
...,...,...,...,...,...,...
2017.05,2017,5,31,2017-05-01,2016,8
2017.06,2017,6,30,2017-06-01,2016,9
2017.07,2017,7,31,2017-07-01,2016,10


## Monthly flow summaries

Add flow summaries to the monthly data

In [9]:
monthly['Flow_min']=daily[['MonthId','Flow']].groupby('MonthId').min()
monthly['Flow_mean']=daily[['MonthId','Flow']].groupby('MonthId').mean()
monthly['Flow_median']=daily[['MonthId','Flow']].groupby('MonthId').median()
monthly['Flow_max']=daily[['MonthId','Flow']].groupby('MonthId').max()
monthly['Volume']=daily[['MonthId','Volume']].groupby('MonthId').sum()
monthly['Flow_range']=monthly['Flow_max']-monthly['Flow_min']
monthly[['Flow_min','Flow_mean','Flow_median','Flow_max','Flow_range']]
monthly

,Year,Month,Days,DateStart,WaterYear,WaterMonth,Flow_min,Flow_mean,Flow_median,Flow_max,Volume,Flow_range
MonthId,,,,,,,,,,,,
1924.10,1924,10,31,1924-10-01,1924,1,111.133124,116.046674,111.133124,126.236260,0.310819,15.103136
1924.11,1924,11,30,1924-11-01,1924,2,127.722788,153.527057,144.946329,191.196189,0.397942,63.473401
1924.12,1924,12,31,1924-12-01,1924,3,192.534369,275.311093,273.832492,438.500503,0.737393,245.966134
1925.01,1925,1,31,1925-01-01,1924,4,418.972890,691.530484,593.052584,1177.094832,1.852195,758.121942
1925.02,1925,2,28,1925-02-01,1924,5,1106.978245,1193.453064,1151.212624,1437.517109,2.887202,330.538864
...,...,...,...,...,...,...,...,...,...,...,...,...
2017.05,2017,5,31,2017-05-01,2016,8,1632.886017,2194.251076,2165.645561,2765.393133,5.877082,1132.507116
2017.06,2017,6,30,2017-06-01,2016,9,673.137147,1117.935234,1121.967762,1609.356985,2.897688,936.219838
2017.07,2017,7,31,2017-07-01,2016,10,381.684831,482.543823,464.672156,642.847453,1.292445,261.162622


## Annual Flow

Create a data table for annual (water year) summaries and populate.

In [10]:
yearly=monthly[['WaterYear']].groupby('WaterYear').count()

yearly['Flow_min']=daily[['WaterYear','Flow']].groupby('WaterYear').min()
yearly['Flow_median']=daily[['WaterYear','Flow']].groupby('WaterYear').median()
yearly['Flow_mean']=daily[['WaterYear','Flow']].groupby('WaterYear').mean()
yearly['Flow_max']=daily[['WaterYear','Flow']].groupby('WaterYear').max()
yearly['Flow_range']=yearly['Flow_max']-yearly['Flow_min']
yearly['Volume']=monthly[['WaterYear','Volume']].groupby('WaterYear').sum()

yearly

,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Volume
WaterYear,,,,,,
1924,111.133124,560.667874,991.317021,3456.928400,3345.795276,31.262174
1925,157.205464,497.238274,1111.041962,4504.008836,4346.803371,35.037819
1926,192.534369,560.667874,990.272631,3305.305419,3112.771050,31.229238
1927,178.800013,504.250658,847.079013,2256.674509,2077.874496,26.786671
1928,195.867340,390.023013,564.051561,1587.035405,1391.168064,17.787930
...,...,...,...,...,...,...
2012,224.058948,694.310091,1336.713717,3617.058785,3392.999837,42.154604
2013,194.855402,809.181755,1340.069290,3578.198961,3383.343558,42.260425
2014,205.608866,538.465854,696.864247,1460.437582,1254.828716,21.976311


In [11]:
Flow_mean_mean=yearly['Flow_mean'].describe()['mean']
Flow_max_mean=yearly['Flow_max'].describe()['mean']
Flow_min_mean=yearly['Flow_min'].describe()['mean']
Volume_mean=yearly['Volume'].describe()['mean']


yearly['Flow_mean_pct_var']=(yearly['Flow_mean']-Flow_mean_mean)/Flow_mean_mean*100
yearly['Flow_max_pct_var']=(yearly['Flow_max']-Flow_max_mean)/Flow_max_mean*100
yearly['Flow_min_pct_var']=(yearly['Flow_min']-Flow_min_mean)/Flow_min_mean*100
yearly['Volume_pct_var']=(yearly['Volume']-Volume_mean)/Volume_mean*100


Flow_mean_mean

1096.0061572902757

In [12]:
yearly['Flow_mean_5yr_mvCoefVar']=yearly['Flow_mean'].rolling(5,center=True).std()/Flow_mean_mean*100
yearly.loc[:,['Flow_mean_pct_var','Volume_pct_var']]

,Flow_mean_pct_var,Volume_pct_var
WaterYear,,
1924,-9.551875,-9.614174
1925,1.371872,1.302049
1926,-9.647165,-9.709399
1927,-22.712203,-22.553836
1928,-48.535731,-48.571178
...,...,...
2012,21.962245,21.878240
2013,22.268409,22.184193
2014,-36.417853,-36.461647


In [14]:
annual_fdcs=pd.DataFrame(index=np.arange(0,1.01,0.01),columns=np.arange(yearly.index.min(),yearly.index.max()+1,1))
for col in annual_fdcs.columns:
    annual_fdcs[col]=np.percentile(daily.loc[daily['WaterYear']==col]['Flow'],((1-annual_fdcs.index)*100))

annual_fdcs

,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0.00,3456.928400,4504.008836,3305.305419,2256.674509,1587.035405,2191.093871,3353.048964,3801.481457,1617.226635,5586.540191,...,3364.446844,5794.676483,5150.675462,4359.719706,3550.600136,3617.058785,3578.198961,1460.437582,2973.367833,3891.623563
0.01,3439.860002,4441.910570,3206.730292,2256.674509,1559.842256,2157.694543,3305.305419,3770.684310,1587.035405,5437.967466,...,3270.839877,5739.508620,5112.322676,4359.719706,3533.387361,3584.091021,3546.121869,1460.437582,2973.367833,3855.250920
0.02,3339.680771,4386.421576,3108.153175,2247.843024,1546.302075,2100.764179,3249.131637,3668.904702,1525.290583,5301.958136,...,3190.576254,5670.600062,5036.576629,4298.973826,3500.658621,3546.300101,3513.271811,1449.924122,2971.029067,3827.656743
0.03,3305.305419,4358.174403,3006.241933,2204.368083,1507.887635,2037.060634,3181.243243,3625.152181,1507.887635,5067.179274,...,3156.299846,5592.029707,5017.027097,4261.290296,3471.033379,3535.242869,3491.391496,1442.943084,2965.571945,3748.419827
0.04,3229.841806,4298.981242,2873.529055,2204.368083,1472.626140,1949.671796,3115.945404,3602.709444,1477.233451,4839.787712,...,3143.067893,5473.090318,4972.542834,4231.108472,3382.520734,3533.644277,3471.545334,1431.923175,2965.571945,3578.267845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.96,111.133124,164.561615,197.600309,182.967257,218.928637,193.886902,218.928637,237.500384,195.867340,170.509341,...,249.694276,195.240229,220.042606,234.093227,254.135758,235.121897,203.102626,234.093227,186.005555,173.346774
0.97,111.133124,164.561615,195.867340,182.967257,203.003584,186.876438,218.312943,237.500384,195.867340,170.509341,...,247.157156,190.780939,216.320970,234.093227,251.382949,231.924494,203.017496,230.611546,182.144524,164.561615
0.98,111.133124,162.710566,195.867340,182.967257,203.003584,186.249402,203.482234,237.500384,195.468577,161.990714,...,244.819005,182.428356,216.320970,232.038664,251.382949,230.611546,198.229576,230.611546,177.515010,164.561615
0.99,111.133124,160.370675,195.468577,181.772295,199.263540,186.005555,203.003584,237.500384,194.080121,160.370675,...,242.696289,180.733596,216.320970,230.611546,243.816940,227.256632,197.067733,224.659583,175.441289,160.370675


In [15]:
yearly['MeanQ3070']=annual_fdcs.loc[(annual_fdcs.index>=0.3) & (annual_fdcs.index<=0.7)].mean()
yearly

,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Volume,Flow_mean_pct_var,Flow_max_pct_var,Flow_min_pct_var,Volume_pct_var,Flow_mean_5yr_mvCoefVar,MeanQ3070
WaterYear,,,,,,,,,,,,
1924,111.133124,560.667874,991.317021,3456.928400,3345.795276,31.262174,-9.551875,-4.440528,-48.269395,-9.614174,NaN,663.867407
1925,157.205464,497.238274,1111.041962,4504.008836,4346.803371,35.037819,1.371872,24.503796,-26.823494,1.302049,NaN,559.189963
1926,192.534369,560.667874,990.272631,3305.305419,3112.771050,31.229238,-9.647165,-8.631824,-10.378482,-9.709399,19.175992,682.846846
1927,178.800013,504.250658,847.079013,2256.674509,2077.874496,26.786671,-22.712203,-37.619007,-16.771594,-22.553836,19.707291,552.440996
1928,195.867340,390.023013,564.051561,1587.035405,1391.168064,17.787930,-48.535731,-56.129764,-8.827040,-48.571178,15.800309,425.609281
...,...,...,...,...,...,...,...,...,...,...,...,...
2012,224.058948,694.310091,1336.713717,3617.058785,3392.999837,42.154604,21.962245,-0.014062,4.295681,21.878240,32.565477,860.975738
2013,194.855402,809.181755,1340.069290,3578.198961,3383.343558,42.260425,22.268409,-1.088260,-9.298080,22.184193,24.733026,978.953448
2014,205.608866,538.465854,696.864247,1460.437582,1254.828716,21.976311,-36.417853,-59.629293,-4.292523,-36.461647,24.308600,592.724205


In [16]:
fdc=pd.DataFrame({'Exceedance': np.arange(0,1.001,0.001)}).set_index('Exceedance')
fdc['Mean']=np.percentile(yearly['Flow_mean'],((1-fdc.index)*100))
fdc['Max']=np.percentile(yearly['Flow_max'],((1-fdc.index)*100))
fdc['Min']=np.percentile(yearly['Flow_min'],((1-fdc.index)*100))
fdc['Median']=np.percentile(yearly['Flow_median'],((1-fdc.index)*100))
fdc['MeanQ3070']=np.percentile(yearly['MeanQ3070'],((1-fdc.index)*100))

fdc

,Mean,Max,Min,Median,MeanQ3070
Exceedance,,,,,
0.000,2333.278712,9911.758819,339.355998,1093.868608,1257.704591
0.001,2316.261238,9841.962369,338.942303,1089.951253,1250.745562
0.002,2299.243764,9772.165919,338.528608,1086.033898,1243.786533
0.003,2282.226289,9702.369469,338.114913,1082.116542,1236.827504
0.004,2265.208815,9632.573019,337.701217,1078.199187,1229.868475
...,...,...,...,...,...
0.996,378.051303,851.088700,105.831249,271.913051,295.294729
0.997,373.450831,844.154523,105.059457,271.481097,293.755590
0.998,368.850358,837.220345,104.287665,271.049143,292.216452


In [17]:
yearly['ExceedanceMean']=pd.merge_asof(yearly.reset_index().sort_values('Flow_mean'),fdc.reset_index().sort_values('Mean'),left_on='Flow_mean',right_on='Mean').set_index('WaterYear')['Exceedance']
yearly['ExceedanceMedian']=pd.merge_asof(yearly.reset_index().sort_values('Flow_median'),fdc.reset_index().sort_values('Median'),left_on='Flow_median',right_on='Median').set_index('WaterYear')['Exceedance']
yearly['ExceedanceMeanQ3070']=pd.merge_asof(yearly.reset_index().sort_values('MeanQ3070'),fdc.reset_index().sort_values('MeanQ3070'),left_on='MeanQ3070',right_on='MeanQ3070').set_index('WaterYear')['Exceedance']
yearly

,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Volume,Flow_mean_pct_var,Flow_max_pct_var,Flow_min_pct_var,Volume_pct_var,Flow_mean_5yr_mvCoefVar,MeanQ3070,ExceedanceMean,ExceedanceMedian,ExceedanceMeanQ3070
WaterYear,,,,,,,,,,,,,,,
1924,111.133124,560.667874,991.317021,3456.928400,3345.795276,31.262174,-9.551875,-4.440528,-48.269395,-9.614174,NaN,663.867407,0.566,0.577,0.490
1925,157.205464,497.238274,1111.041962,4504.008836,4346.803371,35.037819,1.371872,24.503796,-26.823494,1.302049,NaN,559.189963,0.435,0.729,0.740
1926,192.534369,560.667874,990.272631,3305.305419,3112.771050,31.229238,-9.647165,-8.631824,-10.378482,-9.709399,19.175992,682.846846,0.577,0.577,0.457
1927,178.800013,504.250658,847.079013,2256.674509,2077.874496,26.786671,-22.712203,-37.619007,-16.771594,-22.553836,19.707291,552.440996,0.696,0.716,0.772
1928,195.867340,390.023013,564.051561,1587.035405,1391.168064,17.787930,-48.535731,-56.129764,-8.827040,-48.571178,15.800309,425.609281,0.935,0.957,0.946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,224.058948,694.310091,1336.713717,3617.058785,3392.999837,42.154604,21.962245,-0.014062,4.295681,21.878240,32.565477,860.975738,0.250,0.291,0.196
2013,194.855402,809.181755,1340.069290,3578.198961,3383.343558,42.260425,22.268409,-1.088260,-9.298080,22.184193,24.733026,978.953448,0.240,0.149,0.109
2014,205.608866,538.465854,696.864247,1460.437582,1254.828716,21.976311,-36.417853,-59.629293,-4.292523,-36.461647,24.308600,592.724205,0.816,0.613,0.653


## Calendar months

Produce summaries of flow by calendar month

In [18]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly['Flow_min']=daily[['WaterMonth','Flow']].groupby('WaterMonth').min()
calmonthly['Flow_mean']=daily[['WaterMonth','Flow']].groupby('WaterMonth').mean()
calmonthly['Flow_median']=daily[['WaterMonth','Flow']].groupby('WaterMonth').median()
calmonthly['Flow_max']=daily[['WaterMonth','Flow']].groupby('WaterMonth').max()
calmonthly['Flow_std']=daily[['WaterMonth','Flow']].groupby('WaterMonth').std()
calmonthly['Flow_coefvar']=(calmonthly['Flow_std']/calmonthly['Flow_mean']*100).round(1)

calmonthly

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar
WaterMonth,,,,,,,,
1,Oct,10,104.977648,238.595579,230.611546,471.306385,60.224640,25.2
2,Nov,11,102.744081,275.881399,263.127095,641.262334,80.709314,29.3
3,Dec,12,186.876438,431.992745,408.384252,1127.620860,137.994172,31.9
4,Jan,1,273.832492,718.116610,629.767856,3888.545559,346.143364,48.2
5,Feb,2,390.023013,1346.147442,963.050617,9911.758819,1164.527759,86.5
6,Mar,3,504.250658,2529.652693,2172.233613,9529.570904,1656.181136,65.5
7,Apr,4,685.792889,2964.457896,2962.514654,8542.719391,1354.973279,45.7
8,May,5,421.711275,2218.080459,2157.694543,5673.759847,924.785719,41.7
9,Jun,6,273.832492,1201.478405,1127.620860,3926.960162,550.803730,45.8


## Calendar Month Flow exceedance

Flow exceedance values by calendar month.

P90 is flow which is exceeded for 90% of the time.

In [19]:

calmonthly['Flow_P95']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.05)
calmonthly['Flow_P90']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.1)
#calmonthly['Flow_P80']=monthly[['Month','Flow_mean']].groupby('Month').quantile(0.2)
calmonthly['Flow_P75']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.25)
calmonthly['Flow_P50']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.5)
calmonthly['Flow_P25']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.75)
#calmonthly['Flow_P20']=monthly[['Month','Flow_mean']].groupby('Month').quantile(0.8)
calmonthly['Flow_P10']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.9)
calmonthly['Flow_P05']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.95)

calmonthly

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar,Flow_P95,Flow_P90,Flow_P75,Flow_P50,Flow_P25,Flow_P10,Flow_P05
WaterMonth,,,,,,,,,,,,,,,
1,Oct,10,104.977648,238.595579,230.611546,471.306385,60.224640,25.2,163.960680,172.337628,199.372091,228.637711,274.461350,308.102591,343.273895
2,Nov,11,102.744081,275.881399,263.127095,641.262334,80.709314,29.3,176.031597,199.755714,221.011317,266.323478,309.866716,375.815427,409.593174
3,Dec,12,186.876438,431.992745,408.384252,1127.620860,137.994172,31.9,289.626432,308.051521,346.412998,402.301251,503.699189,562.685065,685.355162
4,Jan,1,273.832492,718.116610,629.767856,3888.545559,346.143364,48.2,436.280035,478.628811,542.101989,620.129230,811.344208,996.195907,1314.113257
5,Feb,2,390.023013,1346.147442,963.050617,9911.758819,1164.527759,86.5,626.409663,690.839238,807.233391,986.633624,1417.444938,2478.096386,3201.226914
6,Mar,3,504.250658,2529.652693,2172.233613,9529.570904,1656.181136,65.5,848.541223,909.825644,1184.390848,2339.808237,3120.855022,4345.984890,5638.829815
7,Apr,4,685.792889,2964.457896,2962.514654,8542.719391,1354.973279,45.7,1085.442626,1256.028764,1922.361846,2960.216858,3699.347853,4654.522435,5104.693213
8,May,5,421.711275,2218.080459,2157.694543,5673.759847,924.785719,41.7,876.335485,1128.544360,1682.373622,2234.561289,2673.535307,3215.914788,3623.112951
9,Jun,6,273.832492,1201.478405,1127.620860,3926.960162,550.803730,45.8,450.280286,631.415271,853.629555,1176.988450,1466.071086,1810.876136,1972.049873


## Prepare the Representative Years Summaries

In [20]:
selected = pd.read_csv(input_data + "selected_years.csv").rename(columns={"Year": "WaterYear"}).set_index('WaterYear')
selected['Flow_min']=yearly['Flow_min']
selected['Flow_mean']=yearly['Flow_mean']
selected['Flow_max']=yearly['Flow_max']
selected['Volume']=yearly['Volume']
selected['ExceedanceMean']=yearly['ExceedanceMean']
selected['ExceedanceMedian']=yearly['ExceedanceMedian']
selected['ExceedanceMeanQ3070']=yearly['ExceedanceMeanQ3070']


selected

,Class,Flow_Exceedance,Flow_min,Flow_mean,Flow_max,Volume,ExceedanceMean,ExceedanceMedian,ExceedanceMeanQ3070
WaterYear,,,,,,,,,
1967,Very Wet,Q3,254.145972,1844.708478,5511.046404,58.334109,0.044,0.076,0.022
2013,Wet,Q12,194.855402,1340.069290,3578.198961,42.260425,0.240,0.149,0.109
2002,Median,Q50,194.855402,1072.634961,3847.378538,33.826616,0.500,0.446,0.479
1990,Dry,Q90,197.600309,770.503828,2291.801629,24.298609,0.772,0.914,0.881
1996,Very Dry,Q97,102.744081,546.340570,1398.388034,17.229396,0.946,0.968,0.957


## Save the Data

In [21]:
daily.to_csv(output_data + 'ngonye_flow_daily.csv')
monthly.to_csv(output_data + 'ngonye_flow_monthly.csv')
yearly.to_csv(output_data + 'ngonye_flow_yearly.csv')
calmonthly.to_csv(output_data + 'ngonye_flow_calmonthly.csv')
selected.to_csv(output_data + 'ngonye_flow_selected_years.csv')
fdc.to_csv(output_data + 'ngonye_flow_annual_exceedance.csv')
annual_fdcs.to_csv(output_data + 'ngonye_flow_annual_fdcs.csv')

PermissionError: [Errno 13] Permission denied: './output_data/ngonye_flow_selected_years.csv'